In [ ]:
# Create subset of the data for quicker testing

In [2]:
import os
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import rioxarray as rxr
from rasterio.crs import CRS
from sqlalchemy import create_engine
import rasterstats

In [3]:
password = 'pewp7re'
db_connection_url = "postgresql://postgres:{}@localhost:5432/HSDC".format(password)
con = create_engine(db_connection_url)

In [4]:
pd.set_option('display.max_columns', None)

In [17]:
# Original dataset table names
admin_list = [
    'afg_admbnda_adm2',
    'afg_admbnda_adm1',
    'afg_admbnda_region'
]

flood_list = [
    'afg_fldzonea_100k_ncia_v2_029cm_dissolve_all_v2',
    'afg_fldzonea_100k_ncia_v2_121cm_dissolve_all_v2',
    'afg_fldzonea_100k_ncia_v2_271cm_dissolve_all_v2'
]

# Clip extent to clip by 
clip_extent = gpd.read_file(r'D:\iMMAP\proj\ASDC\data\Flood\clip_extent.shp')

In [ ]:
# Looping over datasets, clips and saves new datasets with _testClip added to table name
def AOI_clipper(dataset_list, clip_extent, db_con, geometry_column_label):
    for i in range(len(dataset_list)):
        dataset = gpd.GeoDataFrame.from_postgis('SELECT * from {}'.format(dataset_list[i]), db_con, geometry_column_label)
        dataset_clipped = clip(dataset, clip_extent)
        dataset_clipped = dataset_clipped.explode() # converting all multipolygons to polygons 
        dataset_clipped.to_postgis('{}_testclip'.format(dataset_list[i]), db_con, if_exists='replace')
        print('{}_testClip - Saved to db'.format(dataset_list[i]))                     

In [ ]:
# Run 
#AOI_clipper(admin_list, clip_extent, con, 'geom')
#AOI_clipper(flood_list, clip_extent, con, 'wkb_geometry')